# Huaibin Ge

In [1]:
import pandas as pd
import numpy as np

In [33]:
data = pd.read_csv("./hw4_data1.csv")

In [34]:
data.head()

,x1,x2
0,-0.201517,-0.683358
1,0.374519,-0.828082
2,-0.161895,-1.247107
3,0.037711,-0.047303
4,-0.260479,1.770204


In [35]:
data = data.values.tolist()

In [36]:
type(data)

list

## a) Implement K-means/median

In [6]:
def K_means_family(datapoints, K, centroid_method, initialization_method, max_iterations):
    
    # initialization_method
    if initialization_method == "Random Split Initialization":
        np.random.shuffle(datapoints)
        Clusters = np.array_split(datapoints, K, axis =0)
        clusters = [cluster.tolist() for cluster in Clusters]
        #print(clusters)
        while max_iterations:
            # calculate centroid
            centroid = []
            new_centroid = []
            for cluster in clusters:
                #print(cluster)
                x = [point[0] for point in cluster]
                y = [point[1] for point in cluster]
                if centroid_method == "Means":
                    center = [np.mean(x), np.mean(y)]
                    centroid.append(center)
                elif centroid_method == "Median":
                    center = [np.median(x), np.median(y)]
                    centroid.append(center)
                    
            # assign piont to the right cluster
            for cluster_number in range(len(clusters)):
                for point in clusters[cluster_number]:
                    dis = abs(np.array(centroid)-np.array(point))
                    distance = [sum(d) for d in dis]
                    #print(distance)
                    belong_cluster, = np.where(distance == min(distance))
                    #print(int(belong_cluster))
                    clusters[cluster_number].remove(point)
                    clusters[int(belong_cluster)].append(point)
                # calcualte new centroid
                x = [point[0] for point in clusters[cluster_number]]
                y = [point[1] for point in clusters[cluster_number]]
                if centroid_method == "Means":
                    center = [np.mean(x), np.mean(y)]
                    new_centroid.append(center)
                elif centroid_method == "Median":
                    center = (np.median(x), np.median(y))
                    new_centroid.append(center)
            if new_centroid == centroid:
                break
            else:
                max_iterations = max_iterations-1
        
    elif initialization_method == "Random Seed Selection Method":
        clusters = []
        np.random.shuffle(datapoints)
        for i in range(K):
            clusters.append([datapoints[i]])
        remain_datapoints = np.delete(datapoints, range(K), axis = 0)
        for point in remain_datapoints:
            distance = []
            for cluster in clusters:
                x = [p[0] for p in cluster]
                y = [p[1] for p in cluster]
                if centroid_method == "Means":
                    center = [np.mean(x), np.mean(y)]
                elif centroid_method == "Median":
                    center = [np.median(x), np.median(y)]
                dis = sum(abs(np.array(center) - np.array(point)))
                distance.append(dis)
            belong, = np.where(min(distance))
            clusters[int(belong)].append(point)
           
    return clusters
        

## b) Silhouette score

In [14]:
def Silhouette(list_of_clusters):
    
    Sx = []
    
    for cluster in list_of_clusters:
        remain_clusters = [other_cluster for other_cluster in list_of_clusters if other_cluster!= cluster]
        for point in cluster:
            distance_in_class = []
            cluster_without_point = [p for p in cluster if p!= point]
        
            
            # distance in class
            for other_point in cluster_without_point:
                d_inclass = sum(abs(np.array(other_point) - np.array(point)))
                distance_in_class.append(d_inclass)
                
            # distance with class
            distance_with_class_record = []
            for remain_cluster in remain_clusters:
                distance_with_class = []
                for remain_point in remain_cluster:
                    d_withclass = sum(abs(np.array(remain_point) - np.array(point)))
                    distance_with_class.append(d_withclass)
                    #print(distance_with_class)
                distance_with_class_record.append(sum(distance_with_class)/len(distance_with_class))
            #print(distance_with_class_record)    
            ax = sum(distance_in_class)/len(distance_in_class)
            bx = min(distance_with_class_record)
            #print(distance_with_class_record)
            #print(bx)
            sx = (bx-ax)/max(ax,bx)
            Sx.append(sx)
    Silhouette_score = sum(Sx)
    
    return Silhouette_score
    

## c) Finding Best K

In [16]:
cluster_2 = K_means_family(data, 2, "Means", "Random Split Initialization",100)

In [17]:
Silhouette(cluster_2)

30.645672726054283

In [18]:
cluster_3 = K_means_family(data, 3, "Means", "Random Split Initialization",100)

In [19]:
Silhouette(cluster_3)

28.980536493338228

In [20]:
cluster_4 = K_means_family(data, 4, "Means", "Random Split Initialization",100)

In [21]:
Silhouette(cluster_4)

25.72326161207576

In [22]:
cluster_5 = K_means_family(data, 5, "Means", "Random Split Initialization",100)

In [23]:
Silhouette(cluster_5)

22.702682091058904

The best K is 2 because it generates largest Silhouette score.

## Bonus: Hierarchical Agglomerative Clustering

In [37]:
import copy

In [67]:
def HAC (datapoint, distance_function):
    datapoint = [[p] for p in datapoint]
    datapoints = copy.deepcopy(datapoint)
    data = copy.deepcopy(datapoint)
    history = []
    diameters = []
    
    def distance_points(x,y):
        dis = sum(abs(np.array(x) - np.array(y)))
        return dis
    
    while len(datapoints) != 1:
        distance = {}
        k = []
        v = []
        for i in range(len(datapoints)):
            for j in range(len(datapoints)):
                if i != j:
                    dis = []
                    for p1 in datapoints[i]:
                        for p2 in datapoints[j]:
                            dis.append(distance_points(p1,p2))
                    #print(dis)
                    if distance_function == "MIN":
                        d = min(dis)
                    elif distance_function == "MAX":
                        d = max(dis)
                    elif distance_function == "AVG":
                        d = np.mean(dis)
                    distance[i,j] = d
        #print(distance)
        i,j = list(distance.keys())[list(distance.values()).index(min(distance.values()))]
        diameters.append(min(distance.values()))
        a = datapoints[i]
        b = datapoints[j]
        m = a+b
        history.append(m)
        datapoints.remove(a)
        datapoints.remove(b) 
        datapoints.append(m)
    
    
    
    cluster_history = data + history
    
    return cluster_history, diameters

In [68]:
cluster_history,diameters = HAC(data, "AVG")

In [69]:
cluster_history

[[[-0.20151729999999998, -0.6833581999999999]],
 [[0.37451947, -0.8280822]],
 [[-0.16189468, -1.2471066000000002]],
 [[0.037710957, -0.047302842000000005]],
 [[-0.26047906, 1.770204]],
 [[0.39217335, 0.7262509]],
 [[-0.595546, -1.4548001]],
 [[-1.2124728999999999, -1.2988458999999999]],
 [[-0.62402, -1.2702229999999999]],
 [[0.5353097, 1.390765]],
 [[1.4814519, 3.5796309999999996]],
 [[1.2141826, 3.5870629999999997]],
 [[3.8623366000000003, 4.2972736]],
 [[4.205326, 4.9409523]],
 [[4.3041506, 5.8463483]],
 [[2.5081396000000002, 4.4804144]],
 [[1.9347998, 3.663017]],
 [[2.211607, 4.263404]],
 [[3.1851852000000003, 4.6804122999999995]],
 [[2.2065609, 3.7271812000000004]],
 [[4.572374, 6.4975176]],
 [[2.637779, 4.8631096]],
 [[4.522715, 6.1088676]],
 [[3.3148953999999997, 6.9001]],
 [[6.754939599999999, 9.133338]],
 [[3.4757779, 6.089272]],
 [[4.326521, 7.821588]],
 [[3.0818818, 5.045526000000001]],
 [[4.10377, 5.597295]],
 [[4.0034175, 5.902063]],
 [[8.99092, 17.243813]],
 [[9.838419, 18

In [70]:
diameters

[0.11722599999999872,
 0.21305110000000005,
 0.27470130000000004,
 0.28421200000000013,
 0.3359253000000004,
 0.35644779999999976,
 0.4272771999999998,
 0.4383090000000003,
 0.46841710000000125,
 0.5123345999999995,
 0.5632932699999997,
 0.7092315500000002,
 0.72076077,
 0.7249730000000003,
 0.7899520000000031,
 0.80765045,
 0.8307581399999998,
 0.8431300000000004,
 0.9317223,
 0.9717105000000004,
 0.9866681000000006,
 0.9964357429999999,
 1.0710257166666672,
 1.349645499999995,
 1.4359166349999999,
 1.4416310000000028,
 1.4957847100000001,
 1.5234876933333334,
 1.5285676750000006,
 1.7967116666666663,
 1.9496654999999992,
 2.097470442857143,
 2.3909430533333333,
 2.938088309,
 3.269294749999999,
 3.6103455647727274,
 7.517545615789474,
 9.2124808055,
 22.237893270166666]

In [71]:
with open('cluster_history.txt', 'w') as f:
    for item in cluster_history:
        f.write("%s\n" % item)

In [72]:
with open('diameters.txt', 'w') as f:
    for item in diameters:
        f.write("%s\n" % item)